In [ ]:
#U-Net for prediction of peak positions

import scipy.io
from scipy.sparse import coo_matrix
import pandas as pd
import matplotlib.pyplot as plt
import time
import cv2
import numpy as np

from sys import getsizeof

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler

from MyDataset import MyDataset
import random

from statsmodels.stats.proportion import proportion_confint

print(torch.__version__)

In [ ]:
torch.set_default_dtype(torch.float64)

images = torch.load('256x256_images_100_percent.pt')
labels = torch.load('256x256_masks_100_percent.pt')

In [ ]:
# number of epochs to train the model
n_epochs = 100
# Learning rate of optimizer
learning_rate = 0.01
# Batch size of data loaders and batch size used when training model
batch_size = 48
#dropout rate
dropout = 0.5

In [ ]:
print(len(images))
print(len(labels))

random.Random(10).shuffle(images) # shuffling with seed
random.Random(10).shuffle(labels) 

#images=images[0:500]
#labels=labels[0:500]

size = len(images)

dataset = MyDataset(images,labels)

split_indices = list(range(0,size))

train_idx=split_indices[0:round(0.70*size)]
val_idx=split_indices[round(0.70*size):round(0.85*size)]
test_idx=split_indices[round(0.85*size):]
print(train_idx)
print(val_idx)
print(test_idx)

train_dataset=MyDataset([images[i] for i in train_idx],[labels[i] for i in train_idx])
val_dataset=MyDataset([images[i] for i in val_idx],[labels[i] for i in val_idx])
test_dataset=MyDataset([images[i] for i in test_idx],[labels[i] for i in test_idx])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
valid_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1)

for data, target in train_loader:
    print(data.shape)
    print(target.shape)
    
for data, target in test_loader:
    print(data.shape)
    print(target.shape)
    

In [ ]:
# define the CNN architecture

class conv_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.conv1 = nn.Conv2d(in_c, out_c, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_c)
        self.conv2 = nn.Conv2d(out_c, out_c, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_c)
        self.relu = nn.ReLU()
            
    def forward(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x
    
class encoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.conv = conv_block(in_c, out_c)
        self.pool = nn.MaxPool2d((2, 2))
    def forward(self, inputs):
        x = self.conv(inputs)
        p = self.pool(x)
        return x, p
    
class decoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_c, out_c, kernel_size=2, stride=2, padding=0)
        self.conv = conv_block(out_c+out_c, out_c)
    def forward(self, inputs, skip):
        x = self.up(inputs)
        x = torch.cat([x, skip], axis=1)
        x = self.conv(x)
        return x

class unet(nn.Module):
    def __init__(self):
        super().__init__()
        """ Encoder """
        self.e1 = encoder_block(1, 16)
        self.e2 = encoder_block(16, 32)
        self.e3 = encoder_block(32, 64)
        self.e4 = encoder_block(64, 128)
        """ Bottleneck """
        self.b = conv_block(128, 256)
        """ Decoder """
        self.d1 = decoder_block(256, 128)
        self.d2 = decoder_block(128, 64)
        self.d3 = decoder_block(64, 32)
        self.d4 = decoder_block(32, 16)
        """ Classifier """
        self.outputs = nn.Conv2d(16, 1, kernel_size=1, padding=0)
        
    def forward(self, inputs):
        """ Encoder """
        #print('inputs:' + str(inputs.shape))
        s1, p1 = self.e1(inputs)
        #print('s1:' + str(s1.shape))
        #print('p1:' + str(p1.shape))
        s2, p2 = self.e2(p1)
        #print('s2:' + str(s2.shape))
        #print('p2:' + str(p2.shape))
        s3, p3 = self.e3(p2)
        #print('s3:' + str(s3.shape))
        #print('p3:' + str(p3.shape))
        s4, p4 = self.e4(p3)
        #print('s4:' + str(s4.shape))
        #print('p4:' + str(p4.shape))
        """ Bottleneck """
        b = self.b(p4)
        #print('b:' + str(b.shape))
        """ Decoder """
        d1 = self.d1(b, s4)
        #print('d1:' + str(d1.shape))
        d2 = self.d2(d1, s3)
        #print('d2:' + str(d2.shape))
        d3 = self.d3(d2, s2)
        #print('d3:' + str(d3.shape))
        d4 = self.d4(d3, s1)
        #print('d4:' + str(d4.shape))
        """ Classifier """
        x = self.outputs(d4)
        outputs = torch.sigmoid(x)
        print('outputs:' + str(outputs.shape))
        return outputs        

In [ ]:
# create a complete CNN
torch.manual_seed(10) # set seed before creating model
model = unet()
        
#print(model.state_dict()['fc1.weight'])

print(model)
    
# specify loss function (Binary cross entropy)
criterion = nn.MSELoss()

# specify optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
#train_on_gpu = False

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    model.cuda()
    criterion.cuda()
    print('CUDA is available!  Training on GPU ...')

valid_loss_min = np.Inf # track change in validation loss

train_loss= [0.0] * n_epochs
valid_loss= [0.0] * n_epochs

for epoch in range(0, n_epochs):

    # keep track of training and validation loss
    #train_loss[epoch] = 0.0
    #valid_loss[epoch] = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        data=data.to_dense() # model needs dense matrices as input
        target=target.to_dense()
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        output = output.to(torch.float64) #
        target = target.to(torch.float64) #
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss[epoch] += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        data=data.to_dense() # model needs dense matrices as input
        target=target.to_dense() # model needs dense matrices as input
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        
        output = output.to(torch.float64) 
        target = target.to(torch.float64) 
        
        #print(output[0][output[0] >= 0.5])
        #print(target[0][target[0] != 0])
        #print(output)
        #print(target)
        #maxOutputs = []
        #for i in range(output.shape[0]):
        #    maxOutputs.append(output[i].max().cpu().detach().numpy() )
        #print(maxOutputs)
        
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss[epoch] += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss[epoch] = train_loss[epoch]/len(train_loader.sampler)
    valid_loss[epoch] = valid_loss[epoch]/len(valid_loader.sampler)
        
    # print training/validation statistics
    print_train_loss=train_loss[epoch]
    print_valid_loss=valid_loss[epoch]
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, print_train_loss, print_valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss[epoch] <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        print_valid_loss))
        torch.save(model.state_dict(), 'my_unet_model.pt')
        valid_loss_min = valid_loss[epoch]

In [ ]:
fig, ax = plt.subplots()
fig.set_dpi(200)
plt.ylim(0,1)
plt.xlabel("Epoch")
plt.ylabel("Loss")
line1 = plt.plot(np.linspace(1, n_epochs, num=n_epochs), train_loss)
line2 = plt.plot(np.linspace(1, n_epochs, num=n_epochs), valid_loss)
plt.legend(["train loss", "valid loss"])
#plt.xticks(np.arange(0, n_epochs, 1))

In [ ]:
#model_1 = unet()
#model_1.load_state_dict(torch.load('my_unet_model_0.001217_sigmoid_BCE_256f_3906d.pt'))
#model_1.eval()

model_2 = unet()
model_2.load_state_dict(torch.load('my_unet_model_0.000310_sigmoid_MSE_256f_3906d.pt'))
model_2.eval()


#if train_on_gpu:
#model.cuda()

#outputList_1 = []
outputList_2 = []
targetList = []
dataList = []


for data, target in test_loader:
        data=data.to_dense() # model needs dense matrices as input
        target=target.to_dense()
        # move tensors to GPU if CUDA is available
        #if train_on_gpu:
        #    data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        #output_1 = model_1(data)
        # calculate the batch loss
        #output_1 = output_1.to(torch.float64) #
        
        output_2 = model_2(data)
        # calculate the batch loss
        output_2 = output_2.to(torch.float64) #
        
        
        target = target.to(torch.float64) #

        #output_1 = output_1.detach().numpy()
        output_2 = output_2.detach().numpy()
        target = target.detach().numpy()
        
        #print(output.shape)
        #print(target.shape)
        
        #outputList_1.append(output_1)
        outputList_2.append(output_2)
        targetList.append(target)
        dataList.append(data)
            

In [ ]:
imageWithCorrectPeak=0
for i in range(len(outputList_2)):
    correctPeak=0
    myTarget=targetList[i]
    myTarget=myTarget.flatten()
    myOutput_2=outputList_2[i]
    myOutput_2=myOutput_2.flatten()
    for j in range(myTarget.size):
        if (myTarget[j] == 1):
            if (myOutput_2[j] > 0.5):
                correctPeak +=1
                break
    if correctPeak > 0:
        imageWithCorrectPeak +=1

print(str(imageWithCorrectPeak/len(outputList_2)))



In [ ]:
#https://machinelearningmastery.com/tour-of-evaluation-metrics-for-imbalanced-classification/
truePositive=0
falsePositive=0
falseNegative=0
total=0
for i in range(len(outputList_2)):
    myTarget=targetList[i]
    myTarget=myTarget.flatten()
    myOutput_2=outputList_2[i]
    myOutput_2=myOutput_2.flatten()
    for j in range(myTarget.size):
        total +=1
        if (myTarget[j] == 1):
            if (myOutput_2[j] >= 0.5):
                truePositive +=1
            else:
                falseNegative +=1
        elif (myOutput_2[j] >= 0.5):
            falsePositive +=1
                
print(str(truePositive))
print(str(falsePositive))
print(str(falseNegative))
print(str(total))

precision = truePositive / (truePositive + falsePositive)
recall = truePositive / (truePositive + falseNegative)
f1_measure = (2 * precision * recall) / (precision + recall)

print(str(precision))
print(str(recall))
print(str(f1_measure))

In [ ]:
image_nr = 97

myTarget=targetList[image_nr]

#myOutput_1=outputList_1[image_nr]

myOutput_2=outputList_2[image_nr]

myData=dataList[image_nr]


#print(myTarget[myTarget != 0.5])
#print(myOutput_1[myOutput_1 >= 0.5])
#print(myOutput_2[myOutput_2 >= 0.5])

myTarget=myTarget.flatten()
#print(myTarget.shape)

#myOutput_1=myOutput_1.flatten()
#print(myOutput.shape)

myOutput_2=myOutput_2.flatten()
#print(myOutput.shape)

#cutOff = min(0.5,myOutput_2.max())
cutOff = 0.5

#myOutput_1[myOutput_1 >= cutOff] = 1
#myOutput_1[myOutput_1 < cutOff] = 0
myOutput_2[myOutput_2 >= cutOff] = 1
myOutput_2[myOutput_2 < cutOff] = 0

#peaks=0
#correctPeak=0
#falsePeak=0
#for j in range(myTarget.size):
#    if (myTarget[j] == 1):
#       peaks += 1
#        if (myOutput_1[j] > 0.5):
#            correctPeak +=1
#    else:
#        if (myOutput_1[j] > 0.5):
#            falsePeak += 1

#print('BCE')
#print('peaks:' + str(peaks))
#print('correctPeak:' + str(correctPeak))
#print('falsePeak:' + str(falsePeak))


peaks=0
correctPeak=0
falsePeak=0
for j in range(myTarget.size):
    if (myTarget[j] == 1):
        peaks += 1
        if (myOutput_2[j] > 0.5):
            correctPeak +=1
    else:
        if (myOutput_2[j] > 0.5):
            falsePeak += 1

print('MSE')
print('peaks:' + str(peaks))
print('correctPeak:' + str(correctPeak))
print('falsePeak:' + str(falsePeak))



ls=np.linspace(1,256,256) 
x_axis=np.tile(ls,256) 
#print(x_axis.shape)
#print(x_axis)

ls=np.linspace(1,256,256) 
y_axis=np.repeat(ls,256) 
#print(y_axis.shape)
#print(y_axis)

fig, ax = plt.subplots()
fig.set_dpi(200)
plt.xlabel('x (m/z)')
plt.ylabel('y (Retention time)')
plt.xlim([-5, 200])
plt.ylim([-5, 240])
plt.tick_params(axis='x', bottom=False, labelbottom=False)
plt.tick_params(axis='y', left=False, labelleft=False)
plt.scatter(x_axis, y_axis, s=myTarget)
ax.set_title('Target')

#fig, ax = plt.subplots()
#fig.set_dpi(200)
#plt.scatter(x_axis, y_axis, s=myOutput_1)
#ax.set_title('Output BCE')

fig, ax = plt.subplots()
fig.set_dpi(200)
plt.xlim([-5, 200])
plt.ylim([-5, 240])
plt.tick_params(axis='x', bottom=False, labelbottom=False)
plt.tick_params(axis='y', left=False, labelleft=False)
plt.xlabel('x (m/z)')
plt.ylabel('y (Retention time)')
plt.scatter(x_axis, y_axis, s=myOutput_2)
ax.set_title('Output') #MSE

fig, ax = plt.subplots()
fig.set_dpi(200)
plt.xlim([-5, 200])
plt.ylim([-5, 240])
plt.tick_params(axis='x', bottom=False, labelbottom=False)
plt.tick_params(axis='y', left=False, labelleft=False)
plt.xlabel('x (m/z)')
plt.ylabel('y (Retention time)')
plt.scatter(x_axis, y_axis, (0.2*myData)**1)
ax.set_title('Intensities')

fig, ax = plt.subplots()
fig.set_dpi(200)
plt.xlim([-5, 200])
plt.ylim([-5, 240])
plt.tick_params(axis='x', bottom=False, labelbottom=False)
plt.tick_params(axis='y', left=False, labelleft=False)
plt.xlabel('x (m/z)')
plt.ylabel('y (Retention time)')
plt.scatter(x_axis, y_axis, (0.2*myData)**1)
plt.scatter(x_axis, y_axis, s=8*myOutput_2, color='k')
ax.set_title('Intensities')

fig, ax = plt.subplots()
fig.set_dpi(200)
plt.xlim([-5, 200])
plt.ylim([-5, 240])
plt.tick_params(axis='x', bottom=False, labelbottom=False)
plt.tick_params(axis='y', left=False, labelleft=False)
plt.xlabel('x (m/z)')
plt.ylabel('y (Retention time)')
plt.scatter(x_axis, y_axis, (0.2*myData)**1)
plt.scatter(x_axis, y_axis, s=8*myOutput_2, color='k')
plt.scatter(x_axis, y_axis, s=1*myTarget, color='r')
ax.set_title('Output')
              
              
                